In [2]:
from scipy import stats
from astropy.table import Table
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import pandas as pd

train = pd.read_csv('data/dummify_fe_jf.csv')
#Create copy for standardization
train_fe = train.copy()

###############################################################################
########################## BOX-COX TRANSFORMATIONS ############################
###############################################################################
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.jarque_bera.html
#http://dataunderthehood.com/2018/01/15/box-cox-transformation-with-python/
def normtesttab(x):
    nm_value, nm_p = stats.normaltest(x)
    jb_value, jb_p = stats.jarque_bera(x)
    data_rows = [('D’Agostino-Pearson', nm_value, nm_p),
                 ('Jarque-Bera', jb_value, jb_p)]
    t = Table(rows=data_rows, names=('Test name', 'Statistic', 'p-value'), 
              meta={'name': 'normal test table'},
          dtype=('S25', 'f8', 'f8'))
    print(t)

normtesttab(train_fe['GarageScore'])
normtesttab(train_fe['TotalSF'])
#normtesttab(train_fe['SalePrice'])

#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.boxcox.html
xt, maxlog, interval = stats.boxcox(train_fe['GarageScore'] + abs(min(train_fe['GarageScore']))+1, alpha=0.05)
print("lambda = {:g}".format(maxlog))
# Power transform with 0.75
train_fe['GarageScore'] = (train_fe['GarageScore'] + abs(min(train_fe['GarageScore']))+1)**0.75

xt, maxlog, interval = stats.boxcox(train_fe['TotalSF'], alpha=0.05)
print("lambda = {:g}".format(maxlog))
# Power transform with 0.25
train_fe['TotalSF'] = (train_fe['TotalSF'] + abs(min(train_fe['TotalSF']))+1)**0.25

xt, maxlog, interval = stats.boxcox(train_fe['SalePrice'], alpha=0.05)
print("lambda = {:g}".format(maxlog))
# Power transform with -0.5
train_fe['SalePrice'] = (train_fe['SalePrice'])**-0.1

## To untransform sale price, we should raise the transformed value to -10


train_fe.to_csv("data/transformed_only_jf.csv", index = False)

C:\Users\Joe Fritch\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\Joe Fritch\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\Joe Fritch\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\Joe Fritch\Anaconda3\lib\site-packages\scipy\stats\stats.py:1408: RuntimeWarning: invalid value encountered in less
  term2 = np.where(denom < 0, term1, np.power((1-2.0/A)/denom, 1/3.0))
C:\Users\Joe Fritch\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0
C:\Users\Joe Fritch\Anaconda3\lib\site-packages\scipy\stats\morestats.py:1029: Runtime

    Test name      Statistic p-value
------------------ --------- -------
D’Agostino-Pearson       nan     nan
       Jarque-Bera       nan     nan
    Test name          Statistic              p-value        
------------------ ------------------ -----------------------
D’Agostino-Pearson  681.2394708041672 1.1768699872361877e-148
       Jarque-Bera 10385.661908986163                     0.0
lambda = 8.47214
lambda = 0.221749
lambda = -0.076924
